# Importing the libraries

In [1]:
import numpy as np
import random
import os
import tensorflow as tf
from tensorflow import keras
import platform
from tensorflow.keras import layers, optimizers, losses, models, Input, Model
import time # Per misurare il tempo di training
from tensorflow.keras.callbacks import EarlyStopping # Per l'early stopping
import matplotlib.pyplot as plt
from tqdm import tqdm # Per mostrare una barra di progresso

# Set seeds for random operations.

In [2]:
# --- 1. Impostazione del Seed Globale all'inizio del tuo script ---
# Questo è il punto chiave per la riproducibilità di TUTTO ciò che segue.
MASTER_RANDOM_SEED = 42
np.random.seed(MASTER_RANDOM_SEED)
random.seed(MASTER_RANDOM_SEED) # Imposta anche il seed per la libreria 'random' di Python se la usi
tf.random.set_seed(MASTER_RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(MASTER_RANDOM_SEED) # Per operazioni basate su hash (es. ordine dei dizionari)
os.environ['TF_DETERMINISTIC_OPS'] = '1' # Forza operazioni deterministiche in TensorFlow 2.x

# Print the HW Specs.

In [3]:
print("--- Dettagli dell'Architettura Hardware della Sessione Colab ---\n")

# --- 1. Dettagli CPU ---
print("--- Dettagli CPU ---")
!lscpu
print("\n")

# --- 2. Dettagli RAM (Memoria) ---
print("--- Dettagli RAM (Memoria) ---")
!cat /proc/meminfo | grep MemTotal
print("\n")

# --- 3. Dettagli Spazio su Disco ---
print("--- Dettagli Spazio su Disco ---")
!df -h /
print("\n")

# --- 4. Dettagli Acceleratore Hardware (GPU/TPU) ---
print("--- Dettagli Acceleratore Hardware (GPU/TPU) ---")
try:
    tpu_address = os.environ.get('COLAB_TPU_ADDR')
    if tpu_address:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        print(f"Tipo Acceleratore: TPU (indirizzo: {tpu_address})")
        print("Dispositivi TPU disponibili:")
        for device in tf.config.list_logical_devices('TPU'):
            print(f"  - {device.name}")
    else:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            print(f"Tipo Acceleratore: GPU")
            for gpu in gpus:
                print(f"  - Dispositivo GPU rilevato: {gpu.name}")
            print("\nDettagli GPU specifici (da `!nvidia-smi`):")
            !nvidia-smi
        else:
            print("Tipo Acceleratore: Nessuna GPU o TPU rilevata (in uso CPU)")

except Exception as e:
    print(f"Si è verificato un errore durante la rilevazione dell'acceleratore: {e}")
    print("Tentativo di rilevare i dispositivi TensorFlow standard:")
    devices = tf.config.list_logical_devices()
    if devices:
        for device in devices:
            print(f"  - Dispositivo rilevato: {device.name}, Tipo: {device.device_type}")
    else:
        print("Nessun dispositivo TensorFlow rilevato.")

print("\n--- Analisi Dettagli Hardware Completata ---")

--- Dettagli dell'Architettura Hardware della Sessione Colab ---

--- Dettagli CPU ---
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.20GHz
    CPU family:           6
    Model:                79
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4399.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
                           ssse3 fma cx16 pcid sse4_1 sse4

# Connect To Gdrive to store the datasets created.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define the paths

In [11]:
# Percorsi dataset
paths = {
    "0-20": {
        "train": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/training_0-20_SNR.npz",
        "val": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_0-20_SNR.npz",
    },
    "11-15": {
        "train": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/training_11-15_SNR.npz",
        "val": "/content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_11-15_SNR.npz",
    }
}

# Directory salvataggio modelli
save_dir = "/content/drive/MyDrive/GitHub/Rayleigh/trained_models/cnn"
os.makedirs(save_dir, exist_ok=True)

# Load the datasets.

In [7]:
def load_dataset(filepath):
    """
    Carica i dati X e y da un file .npz.
    Si aspetta che il file contenga 'X_train' o 'X_val' e 'y_train' o 'y_val'.
    """
    data = np.load(filepath)

    # Controlla se le chiavi per il training sono presenti
    if 'X_train' in data and 'y_train' in data:
        print(f"  Caricato Training data da: {filepath}")
        return data['X_train'], data['y_train']
    # Altrimenti, controlla se le chiavi per la validation sono presenti
    elif 'X_val' in data and 'y_val' in data:
        print(f"  Caricato Validation data da: {filepath}")
        return data['X_val'], data['y_val']
    else:
        # Se nessuna delle combinazioni attese è trovata, solleva un errore
        raise ValueError(f"Il file {filepath} non contiene i dati X e y attesi (né 'X_train'/'y_train' né 'X_val'/'y_val'). "
                         f"Chiavi trovate: {list(data.keys())}")

# CNN model definition

In [8]:
# Definizione CNN con filtri e kernel size variabili
def build_cnn(input_shape, num_filters, kernel_size):
    model_id = f"CNN_{kernel_size}-ksize_{num_filters}-filters"
    inputs = Input(shape=input_shape)
    x = layers.Reshape((input_shape[0], 1))(inputs)
    x = layers.Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = layers.Dropout(0.4)(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = Model(inputs=inputs, outputs=outputs, name=model_id)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model, model_id

# Training...

In [12]:
# Parametri da testare
filter_options = [128, 256, 512, 1024]
kernel_sizes = [3, 5, 7, 9]

# Addestramento su entrambi i range SNR
for snr_range, datasets in paths.items():
    x_train, y_train = load_dataset(datasets['train'])
    x_val, y_val = load_dataset(datasets['val'])

    for filters in filter_options:
        for ksize in kernel_sizes:
            print(f"\n>>> Training CNN con {filters} filtri, kernel size {ksize}, SNR {snr_range}")
            model, model_id = build_cnn(x_train.shape[1:], filters, ksize)

            es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
            model.fit(
                x_train, y_train,
                validation_data=(x_val, y_val),
                epochs=30,
                batch_size=32,
                callbacks=[es],
                verbose=1
            )

            model_filename = f"{model_id}_rayleigh_snr_{snr_range}.h5"
            model.save(os.path.join(save_dir, model_filename))
            print(f"Modello salvato: {model_filename}")

  Caricato Training data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/training_0-20_SNR.npz
  Caricato Validation data da: /content/drive/MyDrive/GitHub/Rayleigh/dataset/validation_0-20_SNR.npz

>>> Training CNN con 128 filtri, kernel size 3, SNR 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.9221 - loss: 0.1629 - val_accuracy: 0.9688 - val_loss: 0.0752
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9651 - loss: 0.0795 - val_accuracy: 0.9718 - val_loss: 0.0643
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9684 - loss: 0.0720 - val_accuracy: 0.9632 - val_loss: 0.0889
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9715 - loss: 0.0645 - val_accuracy: 0.9757 - val_loss: 0.0574
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.9737 - loss: 0.0613 - val_accuracy: 0.9700 - val_loss: 0.0743
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9750 - loss: 0.057

Modello salvato: CNN_3-ksize_128-filters_rayleigh_snr_0-20.h5

>>> Training CNN con 128 filtri, kernel size 5, SNR 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.9400 - loss: 0.1292 - val_accuracy: 0.9791 - val_loss: 0.0516
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9742 - loss: 0.0596 - val_accuracy: 0.9806 - val_loss: 0.0467
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.9770 - loss: 0.0532 - val_accuracy: 0.9802 - val_loss: 0.0452
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9789 - loss: 0.0483 - val_accuracy: 0.9790 - val_loss: 0.0463
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9793 - loss: 0.0474 - val_accuracy: 0.9808 - val_loss: 0.0441
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9813 - loss: 0.0425 - val_accuracy: 0.9812 - val_loss: 0.0419
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9833 - loss: 0.0389 - val

Modello salvato: CNN_5-ksize_128-filters_rayleigh_snr_0-20.h5

>>> Training CNN con 128 filtri, kernel size 7, SNR 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9444 - loss: 0.1188 - val_accuracy: 0.9811 - val_loss: 0.0467
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9779 - loss: 0.0534 - val_accuracy: 0.9822 - val_loss: 0.0423
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9794 - loss: 0.0513 - val_accuracy: 0.9713 - val_loss: 0.0739
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.9800 - loss: 0.0483 - val_accuracy: 0.9831 - val_loss: 0.0422
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - accuracy: 0.9822 - loss: 0.0437 - val_accuracy: 0.9825 - val_loss: 0.0413
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.9835 - loss: 0.0402 - val_accuracy: 0.9822 - val_loss: 0.0408
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9829 - loss: 0.0406 - val

Modello salvato: CNN_7-ksize_128-filters_rayleigh_snr_0-20.h5

>>> Training CNN con 128 filtri, kernel size 9, SNR 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.9490 - loss: 0.1106 - val_accuracy: 0.9814 - val_loss: 0.0437
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9791 - loss: 0.0497 - val_accuracy: 0.9819 - val_loss: 0.0429
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.9813 - loss: 0.0461 - val_accuracy: 0.9829 - val_loss: 0.0385
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.9822 - loss: 0.0424 - val_accuracy: 0.9760 - val_loss: 0.0587
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9825 - loss: 0.0410 - val_accuracy: 0.9826 - val_loss: 0.0391
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.9842 - loss: 0.0374 - val_accuracy: 0.9834 - val_loss: 0.0406
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.9841 - loss: 0.0356 - val

Modello salvato: CNN_9-ksize_128-filters_rayleigh_snr_0-20.h5

>>> Training CNN con 256 filtri, kernel size 3, SNR 0-20
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.9286 - loss: 0.1535 - val_accuracy: 0.9700 - val_loss: 0.0724
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9643 - loss: 0.0801 - val_accuracy: 0.9631 - val_loss: 0.0859
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.9689 - loss: 0.0723 - val_accuracy: 0.9739 - val_loss: 0.0622
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.9716 - loss: 0.0642 - val_accuracy: 0.9740 - val_loss: 0.0604
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9747 - loss: 0.0587 - val_accuracy: 0.9752 - val_loss: 0.0581
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.9760 - loss: 0.0559 - val_accuracy: 0.9728 - val_loss: 0.0605
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.9778 - loss: 0.0511 - 

...

Modello salvato: CNN_9-ksize_512-filters_rayleigh_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 3, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - accuracy: 0.9274 - loss: 0.1556 - val_accuracy: 0.9700 - val_loss: 0.0700
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9680 - loss: 0.0735 - val_accuracy: 0.9722 - val_loss: 0.0620
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9720 - loss: 0.0636 - val_accuracy: 0.9467 - val_loss: 0.1148
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9738 - loss: 0.0595 - val_accuracy: 0.9485 - val_loss: 0.1237
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9750 - loss: 0.0570 - val_accuracy: 0.9666 - val_loss: 0.0741
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 22ms/step - accuracy: 0.9784 - loss: 0.0501 - val_accuracy: 0.9667 - val_loss: 0.1038
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9820 - loss: 0.

Modello salvato: CNN_3-ksize_1024-filters_rayleigh_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 5, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9474 - loss: 0.1167 - val_accuracy: 0.9777 - val_loss: 0.0536
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9756 - loss: 0.0581 - val_accuracy: 0.9797 - val_loss: 0.0476
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9771 - loss: 0.0510 - val_accuracy: 0.9775 - val_loss: 0.0605
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9795 - loss: 0.0467 - val_accuracy: 0.9752 - val_loss: 0.0577
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9815 - loss: 0.0413 - val_accuracy: 0.9778 - val_loss: 0.0572
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9826 - loss: 0.0386 - val_accuracy: 0.9767 - val_loss: 0.0654
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9850 - loss: 0

Modello salvato: CNN_5-ksize_1024-filters_rayleigh_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 7, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 22ms/step - accuracy: 0.9525 - loss: 0.1067 - val_accuracy: 0.9639 - val_loss: 0.0892
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9776 - loss: 0.0525 - val_accuracy: 0.9799 - val_loss: 0.0453
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9796 - loss: 0.0448 - val_accuracy: 0.9776 - val_loss: 0.0650
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9815 - loss: 0.0433 - val_accuracy: 0.9811 - val_loss: 0.0455
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 22ms/step - accuracy: 0.9821 - loss: 0.0397 - val_accuracy: 0.9808 - val_loss: 0.0567
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9837 - loss: 0.0357 - val_accuracy: 0.9787 - val_loss: 0.0502
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9860 - loss: 0

Modello salvato: CNN_7-ksize_1024-filters_rayleigh_snr_11-15.h5

>>> Training CNN con 1024 filtri, kernel size 9, SNR 11-15
Epoch 1/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 44s 22ms/step - accuracy: 0.9559 - loss: 0.1029 - val_accuracy: 0.9797 - val_loss: 0.0489
Epoch 2/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9808 - loss: 0.0449 - val_accuracy: 0.9812 - val_loss: 0.0428
Epoch 3/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9809 - loss: 0.0433 - val_accuracy: 0.9812 - val_loss: 0.0433
Epoch 4/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9823 - loss: 0.0400 - val_accuracy: 0.9800 - val_loss: 0.0480
Epoch 5/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 22ms/step - accuracy: 0.9841 - loss: 0.0371 - val_accuracy: 0.9802 - val_loss: 0.0498
Epoch 6/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9856 - loss: 0.0332 - val_accuracy: 0.9818 - val_loss: 0.0452
Epoch 7/30
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9866 - loss: 0

Modello salvato: CNN_9-ksize_1024-filters_rayleigh_snr_11-15.h5
